In [34]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
headers=['buying','maint','doors','persons','lug_boot','safety','class values']
data=pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/car/car.data',names=headers)
X_orig=data.loc[:,['buying','maint','doors','persons','lug_boot','safety']]
y_orig=data.loc[:,['class values']]

from copy import deepcopy
X = deepcopy(X_orig)
y = deepcopy(y_orig)

In [35]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=42)

print(y_train)

     class values
212         unacc
618         unacc
944           acc
1574        unacc
414         unacc
420         unacc
483         unacc
394           acc
271         unacc
203         unacc
1192        unacc
1663         good
1040          acc
237         unacc
381         unacc
416           acc
261         unacc
67          unacc
1383        unacc
322           acc
124         unacc
1716        unacc
1405        unacc
1216        unacc
554           acc
1516        unacc
1330        unacc
303         unacc
532         unacc
162         unacc
...           ...
276         unacc
955         unacc
1215        unacc
385         unacc
805           acc
1437        unacc
1515        unacc
343         unacc
769         unacc
1332        unacc
1685         good
130         unacc
1687         good
871         unacc
1123          acc
1396        unacc
87          unacc
1482        unacc
330         unacc
1238        vgood
466         unacc
121         unacc
1638        unacc
1044      

In [36]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
encoders = dict() 

for col in headers:
    
    if col=='class values':
        tmp = LabelEncoder()
        y_train[col] = tmp.fit_transform(y_train[col])
        y_test[col] = tmp.transform(y_test[col])
    else:
        X_train[col] = X_train[col].str.strip()
        X_test[col] = X_test[col].str.strip()
        encoders[col] = LabelEncoder()
        X_train[col+'_int'] = encoders[col].fit_transform(X_train[col])
        X_test[col+'_int'] = encoders[col].transform(X_test[col])


D:\anaconda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
D:\anaconda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
D:\anaconda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas

In [37]:
import numpy as np
ss = StandardScaler()
attributes=['buying','maint','doors','persons','lug_boot','safety']
categorical_headers_ints = [x+'_int' for x in attributes]
X_train = ss.fit_transform(X_train[categorical_headers_ints].values).astype(np.float32)
X_test =ss.transform(X_test[categorical_headers_ints].values).astype(np.float32)

#y_train = y_train['class values'].values.astype(np.int)
#y_test = y_test['class values'].values.astype(np.int)

print(X_test)



[[ -1.34631360e+00  -1.33768523e+00   4.48191851e-01  -1.23644519e+00
    7.85222568e-04  -1.23345685e+00]
 [  4.43017721e-01  -4.45321202e-01  -1.34112358e+00  -7.11509353e-03
    7.85222568e-04   1.22397482e+00]
 [ -1.34631360e+00  -1.33768523e+00   1.34284949e+00  -1.23644519e+00
   -1.22023594e+00   1.22397482e+00]
 ..., 
 [ -4.51647937e-01   1.33940673e+00   1.34284949e+00  -7.11509353e-03
   -1.22023594e+00  -1.23345685e+00]
 [  1.33768344e+00   4.47042793e-01   4.48191851e-01  -1.23644519e+00
    1.22180629e+00  -4.74102562e-03]
 [ -1.34631360e+00   4.47042793e-01   1.34284949e+00  -7.11509353e-03
    7.85222568e-04  -1.23345685e+00]]


D:\anaconda\envs\tensorflow\lib\site-packages\sklearn\utils\validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)


In [38]:
import tensorflow as tf
from tensorflow.contrib import learn
from tensorflow.contrib import layers
from tensorflow.contrib.learn.python import SKCompat
from tensorflow.contrib.learn.python.learn.estimators import model_fn as model_fn_lib
tf.logging.set_verbosity(tf.logging.WARN) # control the verbosity of tensor flow


In [39]:
%%time
# we need to tell tensorflow how many inputs to expect and what the data types will be
# for this early example, everything is just numeric, real valued
features_tf = [layers.real_valued_column('', dimension=X_train.shape[1])]
clf = SKCompat(# wrap with SKCompat for easy usage like sklearn
            learn.DNNClassifier(hidden_units=[10], feature_columns=features_tf)
        )

clf.fit(X_train,y_train,steps=100)

Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
Wall time: 3.8 s


In [40]:
from sklearn import metrics as mt

yhat = clf.predict(X_test)
# notice that the output needs some interpretation
# as its not completely the same as sklearn
yhat = yhat['classes']
print(mt.confusion_matrix(y_test,yhat),
      mt.accuracy_score(y_test,yhat))
print(yhat)

[[  0  44   0   0]
 [  0   7   0   0]
 [  0 112   0   0]
 [  0  10   0   0]] 0.0404624277457
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [41]:

%%time
# we can also custimize the classifier somewhat:
clf = SKCompat(# wrap with SKCompat for easy usage like sklearn
            learn.DNNClassifier(hidden_units=[50], 
                                feature_columns=features_tf,
                                activation_fn=tf.nn.sigmoid 
                                # tf.tanh, tf.sigmoid, tf.nn.relu, tf.nn.softmax etc.
                                )
        )

clf.fit(X_train,y_train,steps=1000)

yhat = clf.predict(X_test)['classes']
print(mt.confusion_matrix(y_test,yhat),
      mt.accuracy_score(y_test,yhat))

Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
[[  0  44   0   0]
 [  0   7   0   0]
 [  0 112   0   0]
 [  0  10   0   0]] 0.0404624277457
Wall time: 4.83 s
